In [2]:
import requests,datetime,configparser
from IPython.display import display

In [21]:
# Set defaults
targetData = "sgcn"
dt = datetime.datetime.utcnow().isoformat()
itisGetCommonNamesJSONBaseURL = "https://www.itis.gov/ITISWebService/jsonservice/getAcceptedNamesFromTSN?tsn="
itisGetScientificNameJSONBaseURL = "https://www.itis.gov/ITISWebService/jsonservice/getScientificNameFromTSN?tsn="

In [22]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'/Users/sky/Documents/configs/stuff.py'))

In [23]:
# Build base URL with API key using input the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [24]:
# Retrieve target data
def getSubjectIDs(targetData):
    if targetData == 'sgcn':
        # Build query string to retrieve data
        targetDataSQL = "SELECT DISTINCT sgcn.taxonomicauthorityid_accepted AS subjectid \
            FROM sgcn \
            WHERE sgcn.taxonomicauthorityid_accepted LIKE '%itis%' AND \
            sgcn.scientificname_accepted = ''"
        # Get Data
        targetData = requests.get(getBaseURL()+"&q="+targetDataSQL).json()
        return targetData

In [25]:
# Extract the TSN from the longer "URI-like" string we used for ITIS in the SGCN data
def extractTSN(subjectid):
    tsn = subjectid.replace('http://services.itis.gov/?q=tsn:','')
    return tsn

In [46]:
# Loop through the target data, extract Subject IDs (only ITIS at this point), retrieve data from service, and insert results in tircache
def getITISScientificNameFromTSN(targetData):
    for feature in targetData['features']:
        # Set the subjectid for query and recording
        strSubjectId = feature['properties']['subjectid']

#        itisURL = itisGetScientificNameJSONBaseURL+extractTSN(strSubjectId)
#        itisJSON = requests.get(itisURL).json()
#        q = requests.get(getBaseURL()+"&q=UPDATE sgcn SET scientificname_accepted='"+itisJSON['combinedName']+"' WHERE taxonomicauthorityid_accepted='"+strSubjectId+"'").json()
#        display (q)

        try:
            # Set the URL path to get the common names from TSN via one of the ITIS JSON service end points, get the response JSON, and pull out the common names structure
            itisURL = itisGetScientificNameJSONBaseURL+extractTSN(strSubjectId)
            itisJSON = requests.get(itisURL).json()
            q = requests.get(getBaseURL()+"&q=UPDATE sgcn SET scientificname_accepted='"+itisJSON['combinedName']+"' WHERE taxonomicauthorityid_accepted='"+strSubjectId+"'").json()
            display (q)

        except:
            print ("Problem with "+strSubjectId)

In [47]:
getITISScientificNameFromTSN(getSubjectIDs("sgcn"))